In [ ]:
!pip install --pre higlass-python

In [ ]:
# Config taken from https://github.com/higlass/higlass-pileup
config = {
    "editable": True,
    "trackSourceServers": ["http://higlass.io/api/v1"],
    "exportViewUrl": "/api/v1/viewconfs",
    "views": [
        {
            "initialXDomain": [0, 100000],
            "tracks": {
                "top": [
                    {
                        "type": "pileup",
                        "options": {
                            "axisPositionHorizontal": "right",
                            "axisLabelFormatting": "normal",
                            "showCoverage": True,
                            "colorScale": [
                                "#2c7bb6",
                                "#92c5de",
                                "#ffffbf",
                                "#fdae61",
                                "#808080",
                                "#DCDCDC",
                            ],
                        },
                        "height": 180,
                        "uid": "FylkvVBTSumoJ959HT4-5A",
                        "data": {
                            "type": "bam",
                            "url": "https://pkerp.s3.amazonaws.com/public/bamfile_test/SRR1770413.sorted.bam",
                            "chromSizesUrl": "https://pkerp.s3.amazonaws.com/public/bamfile_test/GCF_000005845.2_ASM584v2_genomic.chrom.sizes",
                            "options": {"maxTileWidth": 30000},
                        },
                        "width": 470,
                    }
                ]
            },
            "layout": {"w": 12, "h": 6, "x": 0, "y": 0},
        }
    ],
}

The above config contains an unknown track type to `hg`, so we get a validation error because we don't know how to render the track!

In [ ]:
import higlass as hg

hg.Viewconf(**config)  # oh no, track not recognized!

# `hg.PluginTrack`

The `hg.PluginTrack` provides a mechanism to hook into the schema validation as well as provide the plugin source for the renderer. The `plugin_url` is a special field which is a `ClassVar` and ignored by pydantic for serde/validation. In `hg` a plugin can by created by subclassing `hg.PluginTrack` and specifying the `type` and `plugin_url`.

In [ ]:
from typing import ClassVar, Literal


class PileupTrack(hg.PluginTrack):
    type: Literal["pileup"]
    plugin_url: ClassVar[str] = (
        "https://unpkg.com/higlass-pileup/dist/higlass-pileup.min.js"
    )


hg.Viewconf[PileupTrack | hg.Track](**config)  # works!

How does this work? The `hg.Viewconf` is a `pydantic.GenericModel` which is _generic_ over the track type. By default, only HiGlass's builtin track types are recognized, so `hg.Viewconf(**data)` will throw an error when a configuration contains an unknown track.

By supplying a our plugin track as a type parameter explicity, `hg.Viewconf[PileupTrack | hg.Track]`, we extend the model to recognize the `PileupTrack` in our config.

This can seem a bit verbose, but supplying the type parameter explicitly is only necessary when deserializing an unknown config, e.g.

```py
hg.Viewconf.parse_file('./pileup-example.json') # error
hg.Viewconf[PileupTrack].parse_file('./pileup-example.json') # works!
```

The `hg.track`, `hg.view`, and `hr.viewconf` utils _infer_ these types so you don't need to supply them!

In [ ]:
from typing import ClassVar, Literal

import higlass as hg

In [ ]:
class MultivecTrack(hg.PluginTrack):
    type: Literal[
        "basic-multiple-line-chart",
        "horizontal-stacked-bar",
        "basic-multiple-bar-chart",
    ]
    plugin_url: ClassVar[str] = (
        "https://unpkg.com/higlass-multivec/dist/higlass-multivec.min.js"
    )

In [ ]:
from typing import ClassVar, Literal

import higlass as hg


class MultivecTrack(hg.PluginTrack):
    type: Literal[
        "basic-multiple-line-chart",
        "horizontal-stacked-bar",
        "basic-multiple-bar-chart",
    ]
    plugin_url: ClassVar[str] = (
        "https://unpkg.com/higlass-multivec/dist/higlass-multivec.min.js"
    )


track = MultivecTrack(
    **{
        "type": "horizontal-stacked-bar",
        "tilesetUid": "my-multivec-db",
        "server": "http://test1.resgen.io/api/v1",
        "height": 200,
        "width": 470,
        "options": {
            "labelPosition": "topLeft",
            "labelColor": "black",
            "labelTextOpacity": 0.4,
            "valueScaling": "exponential",
            "trackBorderWidth": 0,
            "trackBorderColor": "black",
            "heatmapValueScaling": "log",
            "name": "all.KL.bed.multires.mv5",
            "scaledHeight": True,
            "backgroundColor": "white",
            "sortLargestOnTop": True,
        },
    }
)

track2 = PileupTrack(**config["views"][0]["tracks"]["top"][0])

hg.view((track, "top"), (track2, "top"))  # types inferred!

## Extending plugins with `pydantic`

A `type` and `plugin_url` are minimally what is required to implement a plugin track, however, plugin may define additional fields using pydantic models, granting finer control over serde and validation.

Below we define the `data` field on the `SequenceTrack` using a custom pydantic model.

In [ ]:
from pydantic import BaseModel


class SeqeuenceTrackData(BaseModel):
    class Config:
        extra = "forbid"

    type: Literal["fasta"]
    fastaUrl: str
    faiUrl: str
    chromSizesUrl: str


class SequenceTrack(hg.PluginTrack):
    type: Literal["horizontal-sequence"]
    data: SeqeuenceTrackData | None = None
    plugin_url: ClassVar[str] = (
        "https://unpkg.com/higlass-sequence/dist/higlass-sequence.js"
    )


track = SequenceTrack(
    **{
        "uid": "seq_fasta_example",
        "type": "horizontal-sequence",
        "data": {
            "type": "fasta",
            "fastaUrl": "https://aveit.s3.amazonaws.com/higlass/data/sequence/hg38.fa",
            "faiUrl": "https://aveit.s3.amazonaws.com/higlass/data/sequence/hg38.fa.fai",
            "chromSizesUrl": "https://aveit.s3.amazonaws.com/higlass/data/sequence/hg38.mod.chrom.sizes",
        },
        "width": 568,
        "height": 50,
    }
)

track.data  # data is a pydantic model

In [ ]:
track.data.model_dump_json()

In [ ]:
track.data.model_dump()

And the track is faithfully rendered by HiGlass

In [ ]:
hg.view((track, "center"))